In [ ]:
#Preprocessing of CORA dataset
#date : February 2022
#author : Etienne Pauthenet (etienne.pauthenet@gmail.com) & Loic Bachelot (loic.bachelot@gmail.com)
import datetime as dt
import glob
import netCDF4 as nc
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import xarray as xr
from gsw import sigma0

os.getcwd()

# Preselection of the Raw CORA profiles (INSITU_GLO_TS_REP_OBSERVATIONS_013_001_b)
- CORA raw files without moorings MO.nc and drifting buoys DB.nc and TE.nc
- select  profiles that contain PRES, TEMP and PSAL 
- adjusted variable if it exists
- QC = 1 only for the three variables combined TEMP, PSAL and PRES
- Gulf Stream region
- Save the selected profiles in daily xarray dataset and netcdf (TEMP, PSAL, PRES, LON, LAT, JULD (ref to 1950-01-01), DC_REFERENCE, PLATFORM_NUMBER).

In [ ]:
%%time
cora_rep = ''
data_rep = ''
for yy in range(1990,2021):
    listfile = set(glob.glob(cora_rep + str(yy) + '/CO_*.nc')) - set(glob.glob(cora_rep + str(yy) + '/*MO.nc')) - set(glob.glob(cora_rep + str(yy) + '/*TE.nc')) - set(glob.glob(cora_rep + str(yy) + '/*DB.nc'))
    count = list()
    for myfile in listfile:
        filename = os.path.basename(myfile)
        ds = nc.Dataset(myfile, 'r')  # r+
        dm = ds.variables['DATA_MODE'][:]
        dcref = ds.variables['DC_REFERENCE'][:,:]
        lon   = ds.variables['LONGITUDE'][:]
        lat   = ds.variables['LATITUDE'][:]
        juld  = ds.variables['JULD'][:]
        platform_number = ds.variables['PLATFORM_NUMBER'][:]
        ind     = list()
        mydcref = list()
        mypfnum = list()
        for ii in range(0,len(dm)):
            suffix  =''
            #REGION
            if lon[ii]>=-80 and lon[ii]<=-30 and lat[ii]>=23 and lat[ii]<=50:
                #ADJUSTED IF IT EXIST
                if dm[ii] == b'A' or dm[ii] == b'D':
                    suffix = '_ADJUSTED'
                #TEMP AND PSAL AND PRES
                if 'TEMP' in ds.variables.keys() and 'PSAL' in ds.variables.keys() and 'PRES' in ds.variables.keys():
                    ind.append(ii)
                    dc = b''.join(dcref[ii,dcref[ii,:].mask== False])
                    dc = dc.decode('utf-8')
                    mydcref.append(dc)
                    pf = b''.join(platform_number[ii,platform_number[ii,:].mask== False])
                    mypfnum.append(pf)
        if len(ind)>0:
            count.append(len(ind))
            #RETRIEVE QC FOR ind AND CREATE MASK FROM IT
            mask_TEMP = np.isin(ds.variables['TEMP' + suffix  + '_QC'][ind,:].data, b'1')
            mask_PSAL = np.isin(ds.variables['PSAL' + suffix  + '_QC'][ind,:].data, b'1')
            mask_PRES = np.isin(ds.variables['PRES' + suffix  + '_QC'][ind,:].data, b'1')
            #MAKE A SINGLE MASK FROM THE COMBINED THREE OTHERS
            mask = mask_TEMP & mask_PSAL & mask_PRES
            #EXPORT IN DATASET XARRAY
            dsxr = xr.Dataset(
            data_vars=dict(
                TEMP = (["N_PROF","N_PRES"], ma.masked_array(ds.variables['TEMP' + suffix ][ind,:], mask=~mask)),
                PSAL = (["N_PROF","N_PRES"], ma.masked_array(ds.variables['PSAL' + suffix ][ind,:], mask=~mask)),
                PRES = (["N_PROF","N_PRES"], ma.masked_array(ds.variables['PRES' + suffix ][ind,:], mask=~mask)),
                DC_REFERENCE    = (["N_PROF"], mydcref),
                PLATFORM_NUMBER = (["N_PROF"], mypfnum)
            ),
            coords=dict(
                LON  = (["N_PROF"], lon[ind]),
                LAT  = (["N_PROF"], lat[ind]),
                JULD = (["N_PROF"],juld[ind])
            ),
            attrs=dict(description="CORA subsampled : Selection of profiles with (1) TEMP, PSAL and PRES present, (2) adjusted data if adjusted exist and (3) in the Gulf Stream region (lon in [-80,-30], lat in [23,50]"),
            )
            dsxr.JULD.attrs["units"] = "days since 1950-01-01"
            dsxr['DC_REFERENCE'] = dsxr['DC_REFERENCE'].astype('|S8')
            dsxr.to_netcdf(data_rep + myfile[50:76] + "_GulfStream.nc")
        ds.close()
    print('Extracted a total of ' + str(sum(count)) + ' TS profiles in the Gulf Stream Region, for the year ' + str(yy) + '.')

# Interpolate each profiles on a regular vertical grid (All of that is done on the script interp_cora.py)
- Remove data deeper than 1000m
- Interpolate each profiles with np.interp() on 51 depth levels. Fill to the surface with the last non NaN value and fill with NaN to the bottom. Save also a version with NaN at the surface.

In [ ]:
%%time
pi = [0, 1, 2, 3, 4, 5, 6, 8, 10, 12, 14, 16, 19, 22, 26, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 90, 100, 110, 120, 133, 147, 163, 180, 199, 221, 245, 271,
                301, 334, 371, 412, 458, 509, 565, 628, 697, 773, 857, 950, 1000]
listfile = glob.glob('*.nc')
total_files = len(listfile)
datasets = []
nb_prof = 0
count = 0

for myfile in listfile:
    ds = xr.open_dataset(myfile)
    ds = ds.dropna('N_PROF', how = "all", subset=['TEMP', 'PSAL'])
    N = len(ds.N_PROF)
    #Exclude empty files
    if N>0:
        TEMPi = np.empty((N,len(pi)))
        TEMPi[:] = np.nan
        PSALi = np.empty((N,len(pi)))
        PSALi[:] = np.nan
        TEMPs = np.empty((N,len(pi)))
        TEMPs[:] = np.nan
        PSALs = np.empty((N,len(pi)))
        PSALs[:] = np.nan
        for n in range(0,N):
            TEMPi[n,:] = np.interp(pi, ds.PRES[n,:], ds.TEMP[n,:],right=np.nan)
            PSALi[n,:] = np.interp(pi, ds.PRES[n,:], ds.PSAL[n,:],right=np.nan)
            TEMPs[n,:] = np.interp(pi, ds.PRES[n,:], ds.TEMP[n,:],left=np.nan,right=np.nan)
            PSALs[n,:] = np.interp(pi, ds.PRES[n,:], ds.PSAL[n,:],left=np.nan,right=np.nan)
            #EXPORT INTERPOLATED PROFILES
        dsi = xr.Dataset(
            data_vars=dict(
                TEMP_SURF = (["N_PROF","PRES_INTERPOLATED"], TEMPs),
                PSAL_SURF = (["N_PROF","PRES_INTERPOLATED"], PSALs),
                TEMP_INTERP = (["N_PROF","PRES_INTERPOLATED"], TEMPi),
                PSAL_INTERP = (["N_PROF","PRES_INTERPOLATED"], PSALi),
                DC_REFERENCE    = (["N_PROF"], ds.DC_REFERENCE.data),
                PLATFORM_NUMBER = (["N_PROF"], ds.PLATFORM_NUMBER.data)),
            coords=dict(
                LON  = (["N_PROF"], ds.LON.data),
                LAT  = (["N_PROF"], ds.LAT.data),
                PRES_INTERPOLATED  = (["PRES_INTERPOLATED"], pi),
                JULD = (["N_PROF"],ds.JULD.data)))
        datasets.append(dsi)
        nb_prof = nb_prof + len(dsi.N_PROF)
    count = count+1
    if count % 1000 == 0:
        print(f"{(count/total_files)*100}% done")
        print(f"{nb_prof} profiles processed")
        
print('finished, concatenation starting')
ds_interp = xr.concat(datasets, dim='N_PROF')
ds_interp.to_netcdf(data_rep + "ds_interp.nc")

# Cleaning and formating

In [ ]:
ds = xr.open_dataset(data_rep +"ds_interp.nc")
ds = ds.dropna('N_PROF', how = "any", subset=['TEMP_INTERP', 'PSAL_INTERP'])
x = np.array(ds.TEMP_INTERP)
print(f"number of nan left: {np.count_nonzero(np.isnan(x))}")

#Select the period 1993-2019
ds = ds.where(ds.JULD.dt.year<=2019,drop = True)
ds = ds.where(ds.JULD.dt.year>=1993,drop = True)

ds_clean = ds.where(np.isnan(ds['TEMP_SURF'].isel(PRES_INTERPOLATED=14))==False, drop=True)
ds_clean = ds_clean.rename({'LAT': 'LATITUDE', 'LON':'LONGITUDE', 'JULD':'TIME'})
ds_clean.to_netcdf(data_rep + "CORA1000_1993-2019.nc")

# Horizontal interpolation of surface data at the CORA profile's location

In [ ]:
#Load Data
ds = xr.open_dataset(data_rep + "CORA1000_1993-2019.nc")
lats = ds.LATITUDE
lons = ds.LONGITUDE
dates = ds.TIME

#BATHYMETRY
ds_bat = xr.open_dataset('bathymetry_GulfStream.nc')
res_lin = ds_bat.interp(LATITUDE=lats,LONGITUDE=lons,method = 'linear')['bathymetry'].values
ds = ds.assign(variables={"bathy": (('N_PROF'), res_lin)})

#MDT
ds_mdt = xr.open_dataset('mdt-cnes-cls18-global.nc')
ds_mdt = ds_mdt.assign_coords(lon180=(((ds_mdt.longitude + 180) % 360) - 180))  
ds_mdt['longitude'] = ds_mdt.lon180
res = ds_mdt.interp(latitude=lats,longitude=lons,method = 'nearest')['mdt'].values[0]
ds = ds.assign(variables={"MDT": (('N_PROF'), res)})

#SST
path = '/home/datawork-lops-bluecloud/osnet/data_remote_sensing/SST/SST_Gulf_Stream/'
ds_sst = xr.open_mfdataset(path + '/*.nc')
spatial_domain = {"lon":[-80, -30],
                 "lat": [23, 50]}
ds_sst = ds_sst.sel(lon=slice(spatial_domain['lon'][0], spatial_domain['lon'][1]),
                    lat=slice(spatial_domain['lat'][0], spatial_domain['lat'][1]))
res = ds_sst.sel(dict(lat=lats, lon=lons, time=dates), method='nearest')['analysed_sst'].values-273.15
ds = ds.assign(variables={"SST": (('N_PROF'), res)})
res = ds_sst.sel(dict(lat=lats, lon=lons, time=dates), method='nearest')['analysis_uncertainty'].values
ds = ds.assign(variables={"SST_uncertainty": (('N_PROF'), res)})

#Find NaN if any
# In the case of SST, we identified the error being for profiles sampled after 2019-12-31T12:00:00. 
# The xr.interp() function is not assigning properly these profiles and creating NaN. So we change the date of these profiles to 2019-12-31T12:00:00.
sst = np.array(ds.SST_uncertainty)
indna = np.argwhere(np.isnan(sst))[:,0]
indna

if len(indna)>0:
    print(f"{len(indna)} prof to fix")
    lats3  = ds.LATITUDE.isel(N_PROF = indna)
    lons3  = ds.LONGITUDE.isel(N_PROF = indna)
    dates3 = np.datetime64('2019-12-31T12:00:00.000000000')
else:
    print("no prof to fix")

if len(indna)>0:
    res = ds_sst.interp(lat=lats3,lon=lons3,time = dates3,method = 'nearest')['analysed_sst'].values-273.15
    ds.SST[indna] = res
    res = ds_sst.interp(lat=lats3,lon=lons3,time = dates3,method = 'nearest')['analysis_uncertainty'].values
    ds.SST_uncertainty[indna] = res
    ds
    
#SLA
ds_sla = xr.open_mfdataset('*.nc')
ds_sla = ds_sla.assign_coords(lon180=(((ds_sla.longitude + 180) % 360) - 180))  
ds_sla['longitude'] = ds_sla.lon180 
ds_sla = ds_sla.sortby('longitude')
res = ds_sla.sel(dict(latitude=lats, longitude=lons, time=dates), method='nearest')
ds = ds.assign(variables={"SLA": (('N_PROF'), res['sla'].values)}) 
ds = ds.assign(variables={"UGOS": (('N_PROF'), res['ugos'].values)}) 
ds = ds.assign(variables={"VGOS": (('N_PROF'), res['vgos'].values)}) 
ds = ds.assign(variables={"UGOSA": (('N_PROF'), res['ugosa'].values)}) 
ds = ds.assign(variables={"VGOSA": (('N_PROF'), res['vgosa'].values)}) 
ds = ds.assign(variables={"SLA_err": (('N_PROF'), res['err'].values)}) 

#Find NaN if any
#Here we replace by a nearest neighbour  24.875 and -77.625 (The profiles wtih NaN inputs where close to this grid cell)
UGOS = np.array(ds.UGOS)
indna = np.argwhere(np.isnan(UGOS))[:,0]
indna
if len(indna)>0:
    print(f"{len(indna)} prof to fix")
    lats_nan  = 24.875
    lons_nan  = -77.625
    dates_nan = ds.TIME.isel(N_PROF = indna)
else:
    print("no prof to fix")

if len(indna)>0:
    res = ds_sla.sel(dict(latitude=lats_nan, longitude=lons_nan, time=dates_nan), method='nearest')
    ds.UGOS[indna] = res.ugos
    ds.UGOSA[indna] = res.ugosa
    ds.VGOS[indna] = res.vgos
    ds.VGOSA[indna] = res.vgosa

#Check for NaNs
for var in ["UGOS", "UGOSA", "VGOS", "VGOSA"]:
    tmp = np.array(ds[var])
    indna = np.argwhere(np.isnan(tmp))[:,0]
    print(f"var name {var}: {len(indna)}")

#Save
ds.to_netcdf(data_rep + "CORA1000_1993-2019_coloc.nc")

# Rename and add sigma0 and MLD

In [ ]:
#Load Data
ds.to_netcdf(data_rep + "CORA1000_1993-2019_coloc.nc")

# reformating and renaming
ds = ds.rename({"TEMP_INTERP": "TEMP", "PSAL_INTERP": "PSAL"})

SA = gsw.SA_from_SP(ds['PSAL'], ds['PRES_INTERPOLATED'], ds['LONGITUDE'], ds['LATITUDE'])
CT = gsw.CT_from_t(SA, ds['TEMP'], ds['PRES_INTERPOLATED'])
SIG = gsw.sigma0(SA, CT)
ds = ds.assign(variables={"SA": (('N_PROF', 'PRES_INTERPOLATED'), SA.data)})
ds = ds.assign(variables={"CT": (('N_PROF', 'PRES_INTERPOLATED'), CT.data)})
ds = ds.assign(variables={"SIG": (('N_PROF', 'PRES_INTERPOLATED'), SIG.data)})

sig_diff = ds.SIG - ds.SIG.sel(PRES_INTERPOLATED = 10)-0.03
MLD_obs = ds['PRES_INTERPOLATED'].where(sig_diff>0).min(dim='PRES_INTERPOLATED')
ds = ds.assign(MLD = MLD_obs)
ds.to_netcdf(data_rep + "CORA1000_1993-2019_clean.nc")